# Creating a basic chat experience with context variables

In this example, we show how you can build a simple chat bot by sending and updating context with your requests. 

We introduce the Context Variables object which in this demo functions similarly as a key-value store that you can use when running the kernel.

The context is local (i.e. in your computer's RAM) and not persisted anywhere beyond the life of this Jupyter session.

In future examples, we will show how to persist the context on disk so that you can bring it into your applications.  

In this chat scenario, as the user talks back and forth with the bot, the context gets populated with the history of the conversation. During each new run of the kernel, the context can provide the AI with its variables' content. 

In [ ]:
!python -m pip install semantic-kernel==0.3.10.dev0

In [1]:
import semantic_kernel as sk
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion, OpenAIChatCompletion

kernel = sk.Kernel()

useAzureOpenAI = True

# Configure AI service used by the kernel
if useAzureOpenAI:
    deployment, api_key, endpoint = sk.azure_openai_settings_from_dot_env()
    kernel.add_chat_service("chat_completion", AzureChatCompletion(deployment, endpoint, api_key))
else:
    api_key, org_id = sk.openai_settings_from_dot_env()
    kernel.add_chat_service("chat-gpt", OpenAIChatCompletion("gpt-3.5-turbo", api_key, org_id))


Let's define a prompt outlining a dialogue chat bot.

In [3]:
sk_prompt = """
ChatBot can have a conversation with you about any topic.
It can give explicit instructions or say 'I don't know' if it does not have an answer.

{{$history}}
User: {{$user_input}}
ChatBot: """

Register your semantic function

In [4]:
chat_function = kernel.create_semantic_function(sk_prompt, "ChatBot", max_tokens=2000, temperature=0.7, top_p=0.5)

Initialize your context

In [5]:
context = kernel.create_new_context()
context["history"] = ""

Chat with the Bot

In [6]:
context["user_input"] = "Hi, I'm looking for book suggestions"
bot_answer = await chat_function.invoke_async(context=context)
print(bot_answer)

Sure, I'd be happy to help. What genre are you interested in?


Update the history with the output

In [7]:
context["history"] += f"\nUser: {context['user_input']}\nChatBot: {bot_answer}\n"
print(context["history"])


User: Hi, I'm looking for book suggestions
ChatBot: Sure, I'd be happy to help. What genre are you interested in?



Keep Chatting!

In [8]:
async def chat(input_text: str) -> None:
    # Save new message in the context variables
    print(f"User: {input_text}")
    context["user_input"] = input_text

    # Process the user message and get an answer
    answer = await chat_function.invoke_async(context=context)

    # Show the response
    print(f"ChatBot: {answer}")

    # Append the new interaction to the chat history
    context["history"] += f"\nUser: {input_text}\nChatBot: {answer}\n"

In [9]:
await chat("I love history and philosophy, I'd like to learn something new about Greece, any suggestion?")

User: I love history and philosophy, I'd like to learn something new about Greece, any suggestion?
ChatBot: Absolutely, considering your interest in history, philosophy, and Greece, I would recommend "The Histories" by Herodotus, "The Iliad" and "The Odyssey" by Homer for ancient Greek history. For philosophy, you might enjoy "The Republic" by Plato and "Nicomachean Ethics" by Aristotle. If you're interested in a more general overview, "Sailing the Wine-Dark Sea: Why the Greeks Matter" by Thomas Cahill is a great read.


In [10]:
await chat("that sounds interesting, what is it about?")

User: that sounds interesting, what is it about?
ChatBot: "Sailing the Wine-Dark Sea: Why the Greeks Matter" by Thomas Cahill is a book that explores the legacy of the ancient Greeks and their influence on modern society. It covers a wide range of topics, from the Greek's contributions to literature, philosophy, and politics, to their thoughts on love and war. It's a great read if you're interested in understanding why Greek civilization has had such a lasting impact.


In [11]:
await chat("if I read that book, what exactly will I learn about Greek history?")

User: if I read that book, what exactly will I learn about Greek history?
ChatBot: In "Sailing the Wine-Dark Sea: Why the Greeks Matter", you will learn about the profound influence of ancient Greece on modern Western civilization. The book covers various aspects of Greek culture, including their philosophy, politics, art, and literature. You'll gain insights into Greek thought and ideas, their mythology, the Homeric heroes, the Athenian democracy, and the philosophical teachings of Socrates, Plato, and Aristotle. You'll also learn about the Greek's views on warfare and love. Overall, it provides a comprehensive overview of why Greek history and culture are still relevant and influential today.


In [12]:
print(context["history"])


User: Hi, I'm looking for book suggestions
ChatBot: Sure, I'd be happy to help. What genre are you interested in?

User: I love history and philosophy, I'd like to learn something new about Greece, any suggestion?
ChatBot: Absolutely, considering your interest in history, philosophy, and Greece, I would recommend "The Histories" by Herodotus, "The Iliad" and "The Odyssey" by Homer for ancient Greek history. For philosophy, you might enjoy "The Republic" by Plato and "Nicomachean Ethics" by Aristotle. If you're interested in a more general overview, "Sailing the Wine-Dark Sea: Why the Greeks Matter" by Thomas Cahill is a great read.

User: that sounds interesting, what is it about?
ChatBot: "Sailing the Wine-Dark Sea: Why the Greeks Matter" by Thomas Cahill is a book that explores the legacy of the ancient Greeks and their influence on modern society. It covers a wide range of topics, from the Greek's contributions to literature, philosophy, and politics, to their thoughts on love and 

In [13]:
await chat("could you list some more books I could read about this topic?")

User: could you list some more books I could read about this topic?
ChatBot: Of course, here are some more books about Greek history and philosophy:

1. "The Peloponnesian War" by Donald Kagan: This book provides a detailed account of the war between Athens and Sparta.
2. "The Oresteia" by Aeschylus: This is a trilogy of Greek tragedies that is a must-read for anyone interested in Greek literature and mythology.
3. "The Complete Works" by Euripides: Euripides was one of the great playwrights in ancient Greece, and this collection includes all of his surviving plays.
4. "The Greek Way" by Edith Hamilton: This book provides an introduction to Greek culture and thought.
5. "The Birth of Tragedy" by Friedrich Nietzsche: This is a philosophical work that discusses the nature of Greek tragedy.
6. "The Greeks and the Irrational" by E.R. Dodds: This book explores the spiritual and psychological aspects of Greek culture.
7. "Greek Science After Aristotle" by G.E.R. Lloyd: This book covers the d

After chatting for a while, we have built a growing history, which we are attaching to each prompt and which contains the full conversation. Let's take a look!

In [14]:
print(context["history"])


User: Hi, I'm looking for book suggestions
ChatBot: Sure, I'd be happy to help. What genre are you interested in?

User: I love history and philosophy, I'd like to learn something new about Greece, any suggestion?
ChatBot: Absolutely, considering your interest in history, philosophy, and Greece, I would recommend "The Histories" by Herodotus, "The Iliad" and "The Odyssey" by Homer for ancient Greek history. For philosophy, you might enjoy "The Republic" by Plato and "Nicomachean Ethics" by Aristotle. If you're interested in a more general overview, "Sailing the Wine-Dark Sea: Why the Greeks Matter" by Thomas Cahill is a great read.

User: that sounds interesting, what is it about?
ChatBot: "Sailing the Wine-Dark Sea: Why the Greeks Matter" by Thomas Cahill is a book that explores the legacy of the ancient Greeks and their influence on modern society. It covers a wide range of topics, from the Greek's contributions to literature, philosophy, and politics, to their thoughts on love and 